In [8]:
import pandas as pd
import os
import requests
from datetime import datetime, date, timedelta
import time
import pyodbc
conn_str = os.environ['Connection_SQL']

nye = pd.read_csv(f"nye_users_17102024.csv", sep=",", usecols=['key.id'])['key.id'].to_list()

In [12]:
sis_user_ids = []
parametreCanvas = {}
hodeCanvas = {'Authorization': 'Bearer ' + os.environ["tokenCanvas"]}
for user_id in nye:
    url = f"https://hvl.instructure.com/api/v1/users/{user_id}"
    respons = requests.get(url, headers=hodeCanvas, params=parametreCanvas)
    data = respons.json()
    sis_user_ids.append([user_id, data['sis_user_id']])
sud = pd.DataFrame(sis_user_ids)

In [13]:
sud

,0,1
0,109050,fs:203:1313585
1,109049,fs:203:1313377
2,109048,fs:203:1313334


In [3]:
query = """
    MERGE INTO [dbo].[akv_user_id_kobling] AS target 
    USING (VALUES (?, ?)) AS source (user_id, sis_user_id) 
    ON target.[user_id] = source.[user_id]
    WHEN MATCHED THEN
        UPDATE SET target.[sis_user_id] = source.[sis_user_id]
    WHEN NOT MATCHED THEN
        INSERT ([user_id], [sis_user_id]) VALUES (source.[user_id], source.[sis_user_id]);
"""


In [9]:
nye

[109050, 109049, 109048]

In [15]:
with pyodbc.connect(conn_str) as conn:
    cursor = conn.cursor()
    # Upsert the data
    for index, row in sud.iterrows():
        user_id = str(row[0])
        sis_user_id = str(row[1])
        cursor.execute(query, (user_id, sis_user_id))
    conn.commit()